# Valley Bottom Extraction
## Iterate over regional outputs and select input datasets for RVBD tool as well as convert/create necessary datasets.

Create data dictionary to store input parameters for tool.
1. River Network
2. Catchments
3. DEM
4. Wetlands - as feature class or shapefile
5. Output Folder - Directory
6. River Valley Bottom - Output River Valley Bottom Polygon

In [9]:


# Convert wetlands to shapefile
import arcpy
import os
import sys
# Import RVBD modules
import os
from pathlib import Path
import datetime
from data_preparation.RVBD.slope import slope
from data_preparation.RVBD.valleycostdistance import valley
from data_preparation.RVBD.costdistance import costdist

print('imports complete')
print(f'sys paths {sys.path}')

imports complete
sys paths ['C:\\Program Files\\JetBrains\\PyCharm 2021.2.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Program Files\\JetBrains\\PyCharm 2021.2.3\\plugins\\python\\helpers\\pydev', 'C:\\Users\\dwmerrigan\\Documents\\Github\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\Github\\AKSSF', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Program Files\\ArcGIS\\Pro\\bin', 'C:\\Program Files\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Program Files\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages\\futur

In [10]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

In [11]:
# folder containing regional watersheds and covariate source data
datadir = r'D:\\GIS\\AKSSF'
arcpy.env.overwriteOutput = True
# Set data dir equal to directory containing the AKSSF regional sub-folders.
data_dir = r"D:\\GIS\\AKSSF"
arcpy.env.workspace = data_dir
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr
# Regions to be processed
rList = ['Bristol_Bay','Cook_Inlet', 'Copper_River', 'Prince_William_Sound','Kodiak']
regions = arcpy.ListWorkspaces(workspace_type="Folder")
regions = [os.path.normpath(r) for r in regions if os.path.basename(r) in rList]
#working gdb to save intermediate outputs
scratchgdb = r"D:\\GIS\\scratch.gdb"
scratchfol = r"D:\\GIS\\scratch"
rvbd_scratchfol = r"D:\\GIS\\RVBD"
print (regions)


['D:\\GIS\\AKSSF\\Bristol_Bay', 'D:\\GIS\\AKSSF\\Cook_Inlet', 'D:\\GIS\\AKSSF\\Copper_River', 'D:\\GIS\\AKSSF\\Kodiak', 'D:\\GIS\\AKSSF\\Prince_William_Sound']


In [12]:
#Create rvbd scratch folders if they do not already exist

rvbd_scratchfol = r"D:\\GIS\\RVBD_scratch"
for region in regions:
    rname = os.path.basename(region)
    rvbdscratchpath = os.path.join(rvbd_scratchfol, rname + '_RVBD')
    if not arcpy.Exists(rvbdscratchpath):
        os.mkdir(rvbdscratchpath)
        print (f'{rname} does not have a scratch folder')
        print (f'Creating river valley bottom scratch folder at  {rvbdscratchpath}')
    else:
        print(f'Scratch folder for {rname} already created at {rvbdscratchpath}')

    print('----------')


Scratch folder for Bristol_Bay already created at D:\\GIS\\RVBD_scratch\Bristol_Bay_RVBD
----------
Scratch folder for Cook_Inlet already created at D:\\GIS\\RVBD_scratch\Cook_Inlet_RVBD
----------
Scratch folder for Copper_River already created at D:\\GIS\\RVBD_scratch\Copper_River_RVBD
----------
Scratch folder for Kodiak already created at D:\\GIS\\RVBD_scratch\Kodiak_RVBD
----------
Scratch folder for Prince_William_Sound already created at D:\\GIS\\RVBD_scratch\Prince_William_Sound_RVBD
----------


### Create data dictionary to use as input to RVBD script.
 * Collect and convert datasets as necessary to use as inputs for the RVBD script
 * Create region specific output folders if they do not already exist.


In [33]:
rvbdDict = {}
# Walk through gdbs and set input data sets. Convert wetland raster to shapefile if this has not already been done.
for region in regions:
    rname = os.path.basename(region)
    print(rname)
    arcpy.env.workspace = region
    gdb = arcpy.ListWorkspaces(workspace_type='FileGDB')
    outgdb = gdb[0]
    walk = arcpy.da.Walk(region, datatype=['RasterDataset','FeatureClass'])
    rvbdscratchpath = os.path.join(rvbd_scratchfol, rname + '_RVBD')
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            if 'streams_merge' == filename or 'NHDFlowline_merge' == filename:
                strsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,strsource)
            # Set merged watersheds dataset
            elif 'wtds_merge' == filename:
                wtdsource = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,wtdsource)
            # Set elev raster
            elif 'elev.tif' == filename:
                elev_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,elev_path)
            # Set slope raster
            elif 'slope.tif' == filename:
                slope_path = os.path.join(dirpath, filename)
                append_value(rvbdDict,rname,slope_path)
            # Grab wetland raster
            elif filename == 'wetlands.tif':
                print(f'wetlands fc will be saved to {outgdb}')
                print(f'{dirpath},{filename}')
                wras_path = os.path.join(dirpath, filename)
                wrasname = filename[:-4]
                wetfcpath = os.path.join(outgdb, wrasname)
                wetfcpath2 = os.path.join(scratchgdb, wrasname)
                # Make local copy projected in AKAlbers
                if not arcpy.Exists(wetfcpath):
                    print(f'Converting {wrasname} to feature class...')
                    print('----------')
                    wetfc_conv = arcpy.RasterToPolygon_conversion(wras_path,wetfcpath2,raster_field='VALUE',
                                                             simplify='NO_SIMPLIFY',
                                                             create_multipart_features='MULTIPLE_OUTER_PART')
                    wetfc = arcpy.FeatureClassToFeatureClass_conversion(wetfcpath2,outgdb,wrasname,
                                                                        where_clause='gridcode = 1')
                else:
                    print(f'{wetfcpath} already created')
                    wetfc = wetfcpath
                    print('----------')
    append_value(rvbdDict,rname,wetfcpath)
    append_value(rvbdDict,rname,rvbdscratchpath)
    rivbottpath = os.path.join(dirpath + '.gdb',rname + '_riv_valbot')
    append_value(rvbdDict,rname,rivbottpath)
print('----------')
print('Data Dictionary created')

Bristol_Bay
wetlands fc will be saved to D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb
D:\GIS\AKSSF\Bristol_Bay,wetlands.tif
D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wetlands already created
----------
Cook_Inlet
wetlands fc will be saved to D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb
D:\GIS\AKSSF\Cook_Inlet,wetlands.tif
D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wetlands already created
----------
Copper_River
wetlands fc will be saved to D:\GIS\AKSSF\Copper_River\Copper_River.gdb
D:\GIS\AKSSF\Copper_River,wetlands.tif
D:\GIS\AKSSF\Copper_River\Copper_River.gdb\wetlands already created
----------
Kodiak
wetlands fc will be saved to D:\GIS\AKSSF\Kodiak\Kodiak.gdb
D:\GIS\AKSSF\Kodiak,wetlands.tif
D:\GIS\AKSSF\Kodiak\Kodiak.gdb\wetlands already created
----------
Prince_William_Sound
wetlands fc will be saved to D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb
D:\GIS\AKSSF\Prince_William_Sound,wetlands.tif
D:\GIS\AKSSF\Prince_William_Sound\Prince_William_Sound.gdb\wetlands already created
--

In [34]:
#Check paths in dictionary
rvbdDict

{'Bristol_Bay': ['D:\\GIS\\AKSSF\\Bristol_Bay\\elev.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\slope.tif',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\streams_merge',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wtds_merge',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\wetlands',
  'D:\\\\GIS\\\\RVBD_scratch\\Bristol_Bay_RVBD',
  'D:\\GIS\\AKSSF\\Bristol_Bay\\Bristol_Bay.gdb\\Watersheds.gdb\\Bristol_Bay_riv_valbot'],
 'Cook_Inlet': ['D:\\GIS\\AKSSF\\Cook_Inlet\\elev.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\slope.tif',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\NHDFlowline_merge',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wtds_merge',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\wetlands',
  'D:\\\\GIS\\\\RVBD_scratch\\Cook_Inlet_RVBD',
  'D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\Watersheds.gdb\\Cook_Inlet_riv_valbot'],
 'Copper_River': ['D:\\GIS\\AKSSF\\Copper_River\\elev.tif',
  'D:\\GIS\\AKSSF\\Copper_River\\slope.tif',
  'D:\\GIS\\AKSSF\\Copper_River\\Copper_River.

In [35]:
# need to iterate over regions and inputs from data dictionary
for region in regions:
    key = os.path.basename(region)
    print(key)
    dem = rvbdDict[key][0]
    slp_in = rvbdDict[key][1]
    riv = rvbdDict[key][2]
    cat = rvbdDict[key][3]
    wet = rvbdDict[key][4]
    pth = rvbdDict[key][5]
    outval = rvbdDict[key][6]

    print(f'riv = {riv}')
    print(f'cat = {cat}')
    print(f'dem = {dem}')
    print(f'wet = {wet}')
    print(f'pth = {pth}')
    print(f'outval = {outval}')
    print(f'slp_in = {slp_in}')
    print('----------')

Bristol_Bay
riv = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\streams_merge
cat = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wtds_merge
dem = D:\GIS\AKSSF\Bristol_Bay\elev.tif
wet = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\wetlands
pth = D:\\GIS\\RVBD_scratch\Bristol_Bay_RVBD
outval = D:\GIS\AKSSF\Bristol_Bay\Bristol_Bay.gdb\Watersheds.gdb\Bristol_Bay_riv_valbot
slp_in = D:\GIS\AKSSF\Bristol_Bay\slope.tif
----------
Cook_Inlet
riv = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\NHDFlowline_merge
cat = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wtds_merge
dem = D:\GIS\AKSSF\Cook_Inlet\elev.tif
wet = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\wetlands
pth = D:\\GIS\\RVBD_scratch\Cook_Inlet_RVBD
outval = D:\GIS\AKSSF\Cook_Inlet\Cook_Inlet.gdb\Watersheds.gdb\Cook_Inlet_riv_valbot
slp_in = D:\GIS\AKSSF\Cook_Inlet\slope.tif
----------
Copper_River
riv = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\NHDFlowline_merge
cat = D:\GIS\AKSSF\Copper_River\Copper_River.gdb\wtds_merge
dem = D:\GIS\AKSSF\Copper_River\elev.ti

In [ ]:
# Starting script
start = datetime.datetime.now()
print("Script started at %s" % start)
arcpy.AddMessage ("Script started started at %s" % start)
arcpy.AddMessage ("\n")

# Check out spatial extention
arcpy.CheckOutExtension("Spatial")

# Environments
arcpy.env.overwriteOutput = True

# #try with test data
# # Python inputs
# riv =  r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\streams_merge" # river network as a shapefile
# cat = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wtds_merge" # catchments as a shapefile
# dem = r"D:\\GIS\\AKSSF\\Kodiak\\elev.tif"# elevation model as a raster
# wet = r"D:\\GIS\\AKSSF\\Kodiak\\Kodiak.gdb\\wetlands" # measured valley bottom, wetlands or any wet signature in low areas
# pth = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD" # path to outputs (scratch and cost distance folders)
# outval = r"D:\\GIS\\RVBD_scratch\\Kodiak_RVBD\\Kodiak_RVBD.gdb\\kod_rvbd" # path to output river valley
# slp_in = r"D:\\GIS\\AKSSF\\Kodiak\\slope.tif"

# need to iterate over regions and inputs from data dictionary
for region in regions:
    key = os.path.basename(region)
    print(key)
    dem = rvbdDict[key][0]
    slp_in = rvbdDict[key][1]
    riv = rvbdDict[key][2]
    cat = rvbdDict[key][3]
    wet = rvbdDict[key][4]
    pth = rvbdDict[key][5]
    outval = rvbdDict[key][6]

    print(f'riv = {riv}')
    print(f'cat = {cat}')
    print(f'dem = {dem}')
    print(f'wet = {wet}')
    print(f'pth = {pth}')
    print(f'outval = {outval}')
    print(f'slp_in = {slp_in}')
    print('----------')
    # Check to see if scratch geodatabase exists
    s = Path(os.path.join(pth, "outputs.gdb"))
    if s.exists() == False:
        scrg = arcpy.CreateFileGDB_management(pth, "outputs")
        scr = scrg.getOutput(0) # scratch folder
    else:
        scr = os.path.join(pth, "outputs.gdb") #scr = r"in_memory" (if you want to store in memory)

    # Check to see if cost dist folder exists (use code in comments if you want results in a folder instead of gdb)
    c = Path(os.path.join(pth, "costdist.gdb")) # Path(os.path.join(pth, "costdist"))
    if c.exists() == False:
        cstg = arcpy.CreateFileGDB_management(pth, "costdist")  # arcpy.CreateFolder_management(pth, "costdist")
        cst = cstg.getOutput(0) # cost distance folder
    else:
        cst = os.path.join(pth, "costdist.gdb") # os.path.join(pth, "costdist")

    # # Calculate slope and replace 0 slopes with small value - We already have slope rasters so no need for this step
    # slope(riv, cat, dem, scr)
    #
    # Cost distance calculation
    costdist(riv, cat, slp_in, cst, scr)

    # Extract valley
    valley(riv, wet, cst, scr, outval)

# Ending script
end = datetime.datetime.now()
print("Script ended at %s" % end)
arcpy.AddMessage ("Script ended at %s" % end)
arcpy.AddMessage ("\n")
time_elapsed = end - start
print("Time elapsed %s" % (time_elapsed))
arcpy.AddMessage ("Time elapsed %s" % time_elapsed)
arcpy.AddMessage ("\n")
print(".........................................................")
arcpy.AddMessage (".........................................................")
arcpy.AddMessage ("\n")
